In [ ]:
from google.colab import files
files.upload()

KeyboardInterrupt: 

In [ ]:
import pandas as pd
df = pd.read_csv("titanic.csv")

# Mostrar información inicial del DataFrame
print("Datos originales:")
print(df.info())

# Eliminar columnas con más del 50% de valores nulos
threshold = 0.5 * len(df)
df = df.dropna(thresh=threshold, axis=1)

# Completar los valores faltantes (usaremos la media para numéricos y la moda para categóricos)
for col in df.columns:
    if df[col].dtype == "object":  # Datos categóricos
        df[col].fillna(df[col].mode()[0], inplace=True)
    else:  # Datos numéricos
        df[col].fillna(df[col].mean(), inplace=True)

# Mostrar información después de la limpieza
print("\nDatos después de la limpieza:")
print(df.info())


Datos originales:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   float64
 1   survived   1309 non-null   float64
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   float64
 6   parch      1309 non-null   float64
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(7), object(7)
memory usage: 143.4+ KB
None

Datos después de la limpieza:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1310 entries, 0 to 1309
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype 

<ipython-input-2-624b1923a5e6>:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mean(), inplace=True)
<ipython-input-2-624b1923a5e6>:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'd

In [ ]:
from scipy.stats import ks_2samp
import numpy as np

# Identificar las columnas numéricas con valores faltantes antes de la imputación
num_cols = ["age", "fare"]  # Puedes modificar si hubo más columnas con nulos

for col in num_cols:
    original_data = df[col].dropna()  # Valores antes de la imputación
    imputed_data = df[col]  # Valores después de la imputación

    # Prueba de Kolmogorov-Smirnov
    ks_stat, p_value = ks_2samp(original_data, imputed_data)

    print(f"Prueba KS para '{col}':")
    print(f"Estadístico KS: {ks_stat:.4f}, Valor p: {p_value:.4f}")
    if p_value > 0.05:
        print("✅ No hay diferencias significativas en la distribución.\n")
    else:
        print("⚠️ Hay diferencias significativas en la distribución.\n")


Prueba KS para 'age':
Estadístico KS: 0.0000, Valor p: 1.0000
✅ No hay diferencias significativas en la distribución.

Prueba KS para 'fare':
Estadístico KS: 0.0000, Valor p: 1.0000
✅ No hay diferencias significativas en la distribución.



In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Seleccionar las características y la variable objetivo
X = df.drop(columns=["survived", "name", "ticket", "home.dest"])  # Variables predictoras
X = pd.get_dummies(X, drop_first=True)  # Convertir categóricas en numéricas
y = df["survived"].astype(int)  # Convertir a entero (0 y 1)


# Dividir en entrenamiento (75%) y validación (25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Modelo Random Forest con validación cruzada (5 folds)
model = RandomForestClassifier(n_estimators=100, random_state=42)
cv_scores = cross_val_score(model, X_train, y_train, cv=5)

# Entrenar y evaluar el modelo
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Métricas
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
cv_mean_acc = cv_scores.mean()

print(f"Accuracy en entrenamiento: {train_acc:.4f}")
print(f"Accuracy en validación: {test_acc:.4f}")
print(f"Accuracy en validación cruzada: {cv_mean_acc:.4f}")
print(f"Diferencia entre entrenamiento y validación: {abs(train_acc - test_acc):.4f}")


Accuracy en entrenamiento: 0.9705
Accuracy en validación: 0.8018
Accuracy en validación cruzada: 0.7861
Diferencia entre entrenamiento y validación: 0.1686


In [ ]:
# Ajustar hiperparámetros de Random Forest
model = RandomForestClassifier(
    n_estimators=100,  # Número de árboles
    max_depth=10,  # Limitar profundidad de los árboles
    min_samples_split=10,  # Número mínimo de muestras para dividir un nodo
    min_samples_leaf=5,  # Número mínimo de muestras en una hoja
    max_features="sqrt",  # Número máximo de características por división
    random_state=42
)

cv_scores = cross_val_score(model, X_train, y_train, cv=5)

# Entrenar y evaluar el modelo
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Métricas
train_acc = accuracy_score(y_train, y_train_pred)
test_acc = accuracy_score(y_test, y_test_pred)
cv_mean_acc = cv_scores.mean()

print(f"Accuracy en entrenamiento: {train_acc:.4f}")
print(f"Accuracy en validación: {test_acc:.4f}")
print(f"Accuracy en validación cruzada: {cv_mean_acc:.4f}")
print(f"Diferencia entre entrenamiento y validación: {abs(train_acc - test_acc):.4f}")


Accuracy en entrenamiento: 0.8554
Accuracy en validación: 0.7988
Accuracy en validación cruzada: 0.8177
Diferencia entre entrenamiento y validación: 0.0566


In [ ]:
# Instalar XGBoost si no lo tienes
!pip install xgboost

# Importar XGBoost
from xgboost import XGBClassifier

# Crear el modelo XGBoost
model_xgb = XGBClassifier(
    n_estimators=100,  # Número de árboles
    max_depth=5,  # Profundidad máxima de los árboles
    learning_rate=0.05,  # Tasa de aprendizaje
    subsample=0.8,  # Proporción de muestras para cada árbol
    colsample_bytree=0.8,  # Proporción de características por árbol
    random_state=42
)

# Validación cruzada
cv_scores_xgb = cross_val_score(model_xgb, X_train, y_train, cv=5)

# Entrenar el modelo
model_xgb.fit(X_train, y_train)

# Predicciones
y_train_pred_xgb = model_xgb.predict(X_train)
y_test_pred_xgb = model_xgb.predict(X_test)

# Métricas
train_acc_xgb = accuracy_score(y_train, y_train_pred_xgb)
test_acc_xgb = accuracy_score(y_test, y_test_pred_xgb)
cv_mean_acc_xgb = cv_scores_xgb.mean()

print(f"Accuracy en entrenamiento (XGBoost): {train_acc_xgb:.4f}")
print(f"Accuracy en validación (XGBoost): {test_acc_xgb:.4f}")
print(f"Accuracy en validación cruzada (XGBoost): {cv_mean_acc_xgb:.4f}")
print(f"Diferencia entre entrenamiento y validación (XGBoost): {abs(train_acc_xgb - test_acc_xgb):.4f}")


Accuracy en entrenamiento (XGBoost): 0.8717
Accuracy en validación (XGBoost): 0.8018
Accuracy en validación cruzada (XGBoost): 0.8157
Diferencia entre entrenamiento y validación (XGBoost): 0.0699


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV

# Definir el modelo SVM
model_svm = SVC(random_state=42)

# Definir el grid reducido de hiperparámetros
param_grid_svm = {
    'C': [0.1, 1, 10],  # Reducir a solo tres valores
    'kernel': ['linear', 'rbf'],  # Eliminar 'poly' para reducir el tiempo
    'gamma': ['scale'],  # Solo probar 'scale' (más rápido que 'auto')
}

# Inicializar RandomizedSearchCV (n_iter especifica cuántas combinaciones probar)
randomized_search_svm = RandomizedSearchCV(estimator=model_svm, param_distributions=param_grid_svm,
                                           n_iter=20, scoring='accuracy', cv=3, n_jobs=-1, verbose=2, random_state=42)

# Ajustar RandomizedSearchCV
randomized_search_svm.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados: ", randomized_search_svm.best_params_)

# Evaluar el modelo con los mejores parámetros
best_model_svm = randomized_search_svm.best_estimator_

# Predicciones
y_train_pred_svm = best_model_svm.predict(X_train)
y_test_pred_svm = best_model_svm.predict(X_test)

# Métricas
train_acc_svm = accuracy_score(y_train, y_train_pred_svm)
test_acc_svm = accuracy_score(y_test, y_test_pred_svm)
cv_mean_acc_svm = randomized_search_svm.best_score_

print(f"Accuracy en entrenamiento (SVM): {train_acc_svm:.4f}")
print(f"Accuracy en validación (SVM): {test_acc_svm:.4f}")
print(f"Accuracy en validación cruzada (SVM): {cv_mean_acc_svm:.4f}")
print(f"Diferencia entre entrenamiento y validación (SVM): {abs(train_acc_svm - test_acc_svm):.4f}")


Fitting 3 folds for each of 6 candidates, totalling 18 fits


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_search.py:317: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Mejores parámetros encontrados:  {'kernel': 'linear', 'gamma': 'scale', 'C': 0.1}
Accuracy en entrenamiento (SVM): 0.7800
Accuracy en validación (SVM): 0.7805
Accuracy en validación cruzada (SVM): 0.7800
Diferencia entre entrenamiento y validación (SVM): 0.0004


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

# Definir el modelo SVM
model_svm = SVC(random_state=42)

# Ampliar el grid de hiperparámetros (pero sin exagerar)
param_grid_svm = {
    'C': [0.1, 1, 10, 100],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3],  # Solo para 'poly'
    'coef0': [0, 1]  # Ajuste solo para 'poly'
}

# Usar RandomizedSearchCV para un ajuste más rápido (evaluamos 20 combinaciones aleatorias)
randomized_search_svm = RandomizedSearchCV(estimator=model_svm, param_distributions=param_grid_svm,
                                           n_iter=20, scoring='accuracy', cv=3, n_jobs=-1, verbose=2, random_state=42)

# Ajustar RandomizedSearchCV
randomized_search_svm.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados: ", randomized_search_svm.best_params_)

# Evaluar el modelo con los mejores parámetros
best_model_svm = randomized_search_svm.best_estimator_

# Predicciones
y_train_pred_svm = best_model_svm.predict(X_train)
y_test_pred_svm = best_model_svm.predict(X_test)

# Métricas
train_acc_svm = accuracy_score(y_train, y_train_pred_svm)
test_acc_svm = accuracy_score(y_test, y_test_pred_svm)
cv_mean_acc_svm = randomized_search_svm.best_score_

print(f"Accuracy en entrenamiento (SVM): {train_acc_svm:.4f}")
print(f"Accuracy en validación (SVM): {test_acc_svm:.4f}")
print(f"Accuracy en validación cruzada (SVM): {cv_mean_acc_svm:.4f}")
print(f"Diferencia entre entrenamiento y validación (SVM): {abs(train_acc_svm - test_acc_svm):.4f}")


Fitting 3 folds for each of 20 candidates, totalling 60 fits


KeyboardInterrupt: 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

# Definir el modelo de regresión logística
model_lr = LogisticRegression(random_state=42, max_iter=1000)

# Definir el grid de hiperparámetros a ajustar
param_grid_lr = {
    'C': [0.01, 0.1, 1, 10, 100],  # Ajustar regularización
    'solver': ['lbfgs', 'liblinear'],  # Elegir entre diferentes solvers
    'penalty': ['l2'],  # Regularización L2 (común para regresión logística)
}

# Configurar GridSearchCV con validación cruzada
grid_search_lr = GridSearchCV(estimator=model_lr, param_grid=param_grid_lr,
                              scoring='accuracy', cv=5, n_jobs=-1, verbose=2)

# Ajustar el modelo
grid_search_lr.fit(X_train, y_train)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados: ", grid_search_lr.best_params_)

# Evaluar el modelo con los mejores parámetros
best_model_lr = grid_search_lr.best_estimator_

# Predicciones
y_train_pred_lr = best_model_lr.predict(X_train)
y_test_pred_lr = best_model_lr.predict(X_test)

# Métricas
train_acc_lr = accuracy_score(y_train, y_train_pred_lr)
test_acc_lr = accuracy_score(y_test, y_test_pred_lr)
cv_mean_acc_lr = grid_search_lr.best_score_

print(f"Accuracy en entrenamiento (Logistic Regression): {train_acc_lr:.4f}")
print(f"Accuracy en validación (Logistic Regression): {test_acc_lr:.4f}")
print(f"Accuracy en validación cruzada (Logistic Regression): {cv_mean_acc_lr:.4f}")
print(f"Diferencia entre entrenamiento y validación (Logistic Regression): {abs(train_acc_lr - test_acc_lr):.4f}")


Fitting 5 folds for each of 10 candidates, totalling 50 fits
Mejores parámetros encontrados:  {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
Accuracy en entrenamiento (Logistic Regression): 0.7974
Accuracy en validación (Logistic Regression): 0.8018
Accuracy en validación cruzada (Logistic Regression): 0.7933
Diferencia entre entrenamiento y validación (Logistic Regression): 0.0045


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Escalar las características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Definir el modelo de Regresión Logística
logreg_model = LogisticRegression(random_state=42)

# Definir el grid de hiperparámetros a ajustar
param_grid_logreg = {
    'C': [0.02, 0.2, 2, 20],
    'penalty': ['l1', 'l1'],  # L1 es útil si queremos un modelo más esparso
    'solver': ['liblinear', 'saga'],  # 'liblinear' es mejor para datos pequeños y 'saga' para más grandes
}

# Configurar GridSearchCV con validación cruzada
grid_search_logreg = GridSearchCV(estimator=logreg_model, param_grid=param_grid_logreg,
                                  scoring='accuracy', cv=10, n_jobs=-1, verbose=2)

# Ajustar el modelo
grid_search_logreg.fit(X_scaled, y)

# Mostrar los mejores parámetros encontrados
print("Mejores parámetros encontrados: ", grid_search_logreg.best_params_)

# Evaluar el modelo con los mejores parámetros
best_model_logreg = grid_search_logreg.best_estimator_

# Predicciones
y_train_pred_logreg = best_model_logreg.predict(X_scaled)
y_test_pred_logreg = best_model_logreg.predict(X_test)

# Métricas
train_acc_logreg = accuracy_score(y, y_train_pred_logreg)
test_acc_logreg = accuracy_score(y_test, y_test_pred_logreg)
cv_mean_acc_logreg = grid_search_logreg.best_score_

print(f"Accuracy en entrenamiento (Regresión Logística): {train_acc_logreg:.4f}")
print(f"Accuracy en validación (Regresión Logística): {test_acc_logreg:.4f}")
print(f"Accuracy en validación cruzada (Regresión Logística): {cv_mean_acc_logreg:.4f}")
print(f"Diferencia entre entrenamiento y validación (Regresión Logística): {abs(train_acc_logreg - test_acc_logreg):.4f}")


Fitting 10 folds for each of 16 candidates, totalling 160 fits
Mejores parámetros encontrados:  {'C': 0.02, 'penalty': 'l1', 'solver': 'saga'}
Accuracy en entrenamiento (Regresión Logística): 0.7802
Accuracy en validación (Regresión Logística): 0.5884
Accuracy en validación cruzada (Regresión Logística): 0.7802
Diferencia entre entrenamiento y validación (Regresión Logística): 0.1917


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [ ]:
from sklearn.preprocessing import RobustScaler

# Escalar los datos con RobustScaler
scaler_robust = RobustScaler()
X_scaled_robust = scaler_robust.fit_transform(X_train)

# Extender el rango de valores para 'n_neighbors' y ajustar 'weights'
param_grid_balance = {
    'n_neighbors': [5, 7, 9, 11, 13],  # Ajustar 'n_neighbors'
    'weights': ['uniform', 'distance'],  # Pesos uniformes vs. por distancia
    'metric': ['euclidean', 'manhattan', 'chebyshev']  # Probar diferentes métricas
}

# Crear el modelo KNN
knn_balance = KNeighborsClassifier()

# Usar GridSearchCV para encontrar los mejores parámetros
grid_search_balance = GridSearchCV(knn_balance, param_grid_balance, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_balance.fit(X_scaled_robust, y_train)

# Mejor conjunto de parámetros
print("Mejores parámetros encontrados:", grid_search_balance.best_params_)

# Entrenar y evaluar el modelo con los mejores parámetros
best_knn_balance = grid_search_balance.best_estimator_

# Evaluar en el conjunto de entrenamiento y validación
train_accuracy_balance = best_knn_balance.score(X_scaled_robust, y_train)
val_accuracy_balance = best_knn_balance.score(X_test, y_test)

# Validación cruzada
cv_accuracy_balance = cross_val_score(best_knn_balance, X_scaled_robust, y_train, cv=5, scoring='accuracy').mean()

# Mostrar resultados
print(f"Accuracy en entrenamiento (KNN): {train_accuracy_balance:.4f}")
print(f"Accuracy en validación (KNN): {val_accuracy_balance:.4f}")
print(f"Accuracy en validación cruzada (KNN): {cv_accuracy_balance:.4f}")


NameError: name 'KNeighborsClassifier' is not defined

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# Escalar los datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)

# Definir el rango de parámetros a explorar
param_grid_rf = {
    'n_estimators': [100, 200, 300],  # Número de árboles
    'max_depth': [None, 10, 20, 30],  # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],  # Mínimo de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4],  # Mínimo de muestras por hoja
    'bootstrap': [True, False]  # Si se utiliza el muestreo con reemplazo
}

# Crear el clasificador Random Forest
rf = RandomForestClassifier(random_state=42)

# Usar GridSearchCV para encontrar los mejores parámetros
grid_search_rf = GridSearchCV(rf, param_grid_rf, cv=5, scoring='accuracy', n_jobs=-1)
grid_search_rf.fit(X_scaled, y_train)

# Mejor conjunto de parámetros
print("Mejores parámetros encontrados:", grid_search_rf.best_params_)

# Entrenar y evaluar el modelo con los mejores parámetros
best_rf = grid_search_rf.best_estimator_

# Evaluar en el conjunto de entrenamiento y validación
train_accuracy_rf = best_rf.score(X_scaled, y_train)
val_accuracy_rf = best_rf.score(X_test, y_test)

# Validación cruzada
cv_accuracy_rf = cross_val_score(best_rf, X_scaled, y_train, cv=5, scoring='accuracy').mean()

# Mostrar resultados
print(f"Accuracy en entrenamiento (Random Forest): {train_accuracy_rf:.4f}")
print(f"Accuracy en validación (Random Forest): {val_accuracy_rf:.4f}")
print(f"Accuracy en validación cruzada (Random Forest): {cv_accuracy_rf:.4f}")


Mejores parámetros encontrados: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Accuracy en entrenamiento (Random Forest): 0.8605
Accuracy en validación (Random Forest): 0.5884
Accuracy en validación cruzada (Random Forest): 0.8167
